In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

/Users/mohammedpathariya/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load landmark dataset
df = pd.read_csv('landmarks_dataset.csv')

print("Shape of Data:", df.shape)
df.head()

Shape of Data: (180, 64)


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,label
0,0.517754,0.905560,1.473566e-07,0.516693,0.798606,0.019614,0.496096,0.718364,0.033674,0.457531,...,0.361810,0.815343,0.035725,0.342632,0.790637,0.044646,0.328020,0.768959,0.050737,please
1,0.584487,0.921963,2.834070e-07,0.582382,0.820848,0.008208,0.568748,0.744881,0.010880,0.539280,...,0.422817,0.880903,-0.015751,0.397436,0.864309,-0.016621,0.377107,0.845363,-0.016414,please
2,0.425893,0.868665,1.512977e-07,0.430534,0.772791,0.008915,0.454295,0.715321,0.012780,0.489662,...,0.566774,0.838963,-0.006008,0.586781,0.826357,-0.001663,0.601068,0.814274,0.002199,please
3,0.559062,0.944716,7.708704e-08,0.549858,0.845902,0.007825,0.564966,0.768869,0.013401,0.591337,...,0.703882,0.871311,0.008180,0.720365,0.844001,0.014023,0.730470,0.820380,0.019484,please
4,0.518579,0.941244,9.775813e-08,0.511786,0.845036,0.004552,0.525681,0.768467,0.006787,0.551321,...,0.662779,0.862471,-0.004726,0.678994,0.835751,-0.000824,0.688797,0.811928,0.003094,please


In [3]:
X = df.drop('label', axis=1)  # Landmark features
y = df['label']               # Target labels

In [4]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Optional: To check mapping
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label Mapping:", label_mapping)

# Save the label mapping for future use (real-time prediction)
np.save('label_classes.npy', le.classes_)

Label Mapping: {'goodbye': 0, 'hello': 1, 'no': 2, 'please': 3, 'thank_you': 4, 'yes': 5}


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print("Train Set Shape:", X_train.shape)
print("Test Set Shape:", X_test.shape)

Train Set Shape: (144, 63)
Test Set Shape: (36, 63)


In [6]:
rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)

print("Model Training Completed!")

Model Training Completed!


In [7]:
y_pred = rfc.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9166666666666666

Classification Report:
               precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      1.00      1.00         6
           2       0.86      1.00      0.92         6
           3       1.00      0.83      0.91         6
           4       0.83      0.83      0.83         6
           5       1.00      0.83      0.91         6

    accuracy                           0.92        36
   macro avg       0.92      0.92      0.92        36
weighted avg       0.92      0.92      0.92        36


Confusion Matrix:
 [[6 0 0 0 0 0]
 [0 6 0 0 0 0]
 [0 0 6 0 0 0]
 [0 0 0 5 1 0]
 [0 0 1 0 5 0]
 [1 0 0 0 0 5]]


In [9]:
joblib.dump(rfc, 'rfc_model.pkl')
print("Model saved as 'rfc_model.pkl'")

Model saved as 'rfc_model.pkl'


In [11]:
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train, y_train)

SVC(probability=True)

In [12]:
y_pred_svm = svm_model.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))
print(confusion_matrix(y_test, y_pred_svm))

SVM Accuracy: 0.8333333333333334
              precision    recall  f1-score   support

           0       0.56      0.83      0.67         6
           1       1.00      1.00      1.00         6
           2       0.67      0.33      0.44         6
           3       1.00      1.00      1.00         6
           4       0.86      1.00      0.92         6
           5       1.00      0.83      0.91         6

    accuracy                           0.83        36
   macro avg       0.85      0.83      0.82        36
weighted avg       0.85      0.83      0.82        36

[[5 0 0 0 1 0]
 [0 6 0 0 0 0]
 [4 0 2 0 0 0]
 [0 0 0 6 0 0]
 [0 0 0 0 6 0]
 [0 0 1 0 0 5]]


In [13]:
joblib.dump(svm_model, 'svm_model.pkl')
print("SVM Model saved!")

SVM Model saved!


In [14]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [15]:
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

XGBoost Accuracy: 0.8611111111111112
              precision    recall  f1-score   support

           0       0.83      0.83      0.83         6
           1       0.75      1.00      0.86         6
           2       1.00      1.00      1.00         6
           3       0.83      0.83      0.83         6
           4       0.83      0.83      0.83         6
           5       1.00      0.67      0.80         6

    accuracy                           0.86        36
   macro avg       0.88      0.86      0.86        36
weighted avg       0.88      0.86      0.86        36

[[5 0 0 1 0 0]
 [0 6 0 0 0 0]
 [0 0 6 0 0 0]
 [0 0 0 5 1 0]
 [0 1 0 0 5 0]
 [1 1 0 0 0 4]]


In [16]:
joblib.dump(xgb_model, 'xgb_model.pkl')
print("XGBoost Model saved!")

XGBoost Model saved!


In [17]:
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000)
mlp_model.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000)

In [18]:
y_pred_mlp = mlp_model.predict(X_test)

print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp))
print(confusion_matrix(y_test, y_pred_mlp))

MLP Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         6
           3       1.00      1.00      1.00         6
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         6

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

[[6 0 0 0 0 0]
 [0 6 0 0 0 0]
 [0 0 6 0 0 0]
 [0 0 0 6 0 0]
 [0 0 0 0 6 0]
 [0 0 0 0 0 6]]


In [19]:
joblib.dump(mlp_model, 'mlp_model.pkl')
print("MLP Model saved!")

MLP Model saved!
